In [10]:
import altair as alt
import ibis_vega_transform
import ibis.omniscidb


conn = ibis.omniscidb.connect(
    host='metis.mapd.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)

In [11]:
t = conn.table("flights_donotmodify")

In [12]:
t.head().execute()

,flight_year,flight_month,flight_dayofmonth,flight_dayofweek,deptime,crsdeptime,arrtime,crsarrtime,uniquecarrier,flightnum,...,dest_name,dest_city,dest_state,dest_country,dest_lat,dest_lon,origin_merc_x,origin_merc_y,dest_merc_x,dest_merc_y
0,2008,1,31,4,1039,1045,1258,1307,XE,2592,...,Indianapolis International,Indianapolis,IN,USA,39.717331,-86.294388,-8256417.5,4967090.5,-9606247.0,4824950.0
1,2008,1,5,6,1713,1712,2018,2010,XE,2227,...,George Bush Intercontinental,Houston,TX,USA,29.980473,-95.339722,-10376860.0,5602735.5,-10613169.0,3501040.0
2,2008,1,19,6,1524,1530,1652,1715,XE,1291,...,Newark Intl,Newark,NJ,USA,40.692497,-74.168663,-7904260.5,5215714.0,-8256417.5,4967090.5
3,2008,1,31,4,1526,1525,1742,1740,XE,3067,...,Memphis International,Memphis,TN,USA,35.042416,-89.976669,-8256417.5,4967090.5,-10016157.0,4169647.0
4,2008,1,7,1,1842,1850,2057,2125,XE,2379,...,Louisville International-Standiford,Louisville,KY,USA,38.174389,-85.736000,-8256417.5,4967090.5,-9544088.0,4604090.5


In [2]:

states = alt.selection_multi(fields=['origin_state'])
airlines = alt.selection_multi(fields=['carrier_name'])

dates = alt.selection_interval(
    fields=['dep_timestamp'],
    encodings=['x'],
)

HEIGHT = 800
WIDTH = 1000

count_filter = alt.Chart(
    t[t.dep_timestamp, t.depdelay, t.origin_state, t.carrier_name],
    title="Selected Rows"
).transform_filter(
    airlines
).transform_filter(
    dates
).transform_filter(
    states
).mark_text().encode(
    text='count()'
)

count_total = alt.Chart(
    t,
    title="Total Rows"
).mark_text().encode(
    text='count()'
)

flights_by_state = alt.Chart(
    t[t.origin_state, t.carrier_name, t.dep_timestamp],
    title="Total Number of Flights by State"
).transform_filter(
    airlines
).transform_filter(
    dates
).mark_bar().encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(
    states
).properties(
    height= 2 * HEIGHT / 3,
    width=WIDTH / 2
) + alt.Chart(
    t[t.origin_state, t.carrier_name, t.dep_timestamp],
).transform_filter(
    airlines
).transform_filter(
    dates
).mark_text(dx=20).encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    text='count()'
).properties(
    height= 2 * HEIGHT / 3,
    width=WIDTH / 2
)

carrier_delay = alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state, t.dep_timestamp],
    title="Carrier Departure Delay by Arrival Delay (Minutes)"
).transform_filter(
    states
).transform_filter(
    dates
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_point(filled=True, size=200).encode(
    x='depdelay',
    y='arrdelay',
    color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['carrier_name', 'depdelay', 'arrdelay']
).add_selection(
    airlines
).properties(
    height=2 * HEIGHT / 3,
    width=WIDTH / 2
) + alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state, t.dep_timestamp],
).transform_filter(
    states
).transform_filter(
    dates
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_text().encode(
    x='depdelay',
    y='arrdelay',
    text='carrier_name',
).properties(
    height=2 * HEIGHT / 3,
    width=WIDTH / 2
)

time = alt.Chart(
    t[t.dep_timestamp, t.depdelay, t.origin_state, t.carrier_name],
    title='Number of Flights by Departure Time'
).transform_filter(
    'datum.dep_timestamp != null'
).transform_filter(
    airlines
).transform_filter(
    states
).mark_line().encode(
    alt.X(
        'yearmonthdate(dep_timestamp):T',
    ),
    alt.Y(
        'count():Q',
        scale=alt.Scale(zero=False)
    )
).add_selection(
    dates
).properties(
    height=HEIGHT / 3,
    width=WIDTH + 50
)

(
    (count_filter | count_total) &
    (flights_by_state | carrier_delay) &
    time
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
)

alt.VConcatChart(...)